In [2]:
import os

In [3]:
%pwd

'/Users/kunal/Developer/DL Project/Chicken-Disease-Classification-Using-Fecal-Image/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/kunal/Developer/DL Project/Chicken-Disease-Classification-Using-Fecal-Image'

- For model evaluation we don't need configuration hence we will not update config.yaml file

In [6]:
import dagshub
dagshub.init(repo_owner='bappymalik4161', repo_name='ML-Pipeline-using-DVC', mlflow=True)

ModuleNotFoundError: No module named 'dagshub'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    #mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json # I am importing save_json from utility because I will be saving the evaluation metrics

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            #mlflow_uri="https://dagshub.com/bappymalik4161/ML-Pipeline-using-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [10]:
import tensorflow as tf
#import mlflow
#import mlflow.keras
from urllib.parse import urlparse

In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    # Automatically generate validation data for the model evaluation purpose
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    # This function will load the model
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    # Evaluation function
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model) # first it will load the model
        self._valid_generator() # generate validation data
        self.score = model.evaluate(self.valid_generator) # at last it will perform the evaluation and I will save the score object in a score variable

    # This function will return the score
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]} # extracting loss and accuracy
        save_json(path=Path("scores.json"), data=scores) # saving loss and accuracy in a json file

    
'''
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

'''

'\n    def log_into_mlflow(self):\n        mlflow.set_registry_uri(self.config.mlflow_uri)\n        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme\n        \n        with mlflow.start_run():\n            mlflow.log_params(self.config.all_params)\n            mlflow.log_metrics(\n                {"loss": self.score[0], "accuracy": self.score[1]}\n            )\n            # Model registry does not work with file store\n            if tracking_url_type_store != "file":\n\n                # Register the model\n                # There are other ways to use the Model Registry, which depends on the use case,\n                # please refer to the doc for more information:\n                # https://mlflow.org/docs/latest/model-registry.html#api-workflow\n                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")\n            else:\n                mlflow.keras.log_model(self.model, "model")\n\n'

In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    #evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-03-01 09:49:09,274: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-01 09:49:09,276: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-01 09:49:09,277: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 10s 1s/step - loss: 0.2766 - accuracy: 0.9741
[2025-03-01 09:49:19,080: INFO: common: json file saved at: scores.json]
